Copyright **`(c)`** 2023 Thomas Baracco `<s308722@studenti.polito.it>`  
[`https://github.com/baraccothomas/computational-intelligence`](https://github.com/baraccothomas/computational-intelligence)  
Free for personal or classroom use.